<a href="https://colab.research.google.com/github/KolipakaRamesh/AIML_Practice_Excercises/blob/main/WeatherPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install dependencies
!pip install meteostat pandas scikit-learn joblib fastapi uvicorn pyngrok streamlit

  Using cached meteostat-1.7.4-py3-none-any.whl.metadata (4.6 kB)
  Using cached pyngrok-7.3.0-py3-none-any.whl.metadata (8.1 kB)
  Using cached streamlit-1.48.1-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [6]:
# Download weather data
from meteostat import Daily, Point
import pandas as pd
from datetime import datetime

# Example city: Hyderabad, India (change lat/long if needed)
location = Point(17.3850, 78.4867)

start = datetime(2015, 1, 1)
end = datetime(2024, 12, 31)

# Fetch data
data = Daily(location, start, end)
df = data.fetch()

# Keep useful columns
df = df[["tavg", "tmin", "tmax", "prcp"]]
df.to_csv("weather.csv")

print("✅ Weather data saved as weather.csv")
df.head()

✅ Weather data saved as weather.csv


,tavg,tmin,tmax,prcp
time,,,,
2015-01-03,24.2,<NA>,30.4,0.5
2015-01-04,23.3,18.8,30.4,0.0
2015-01-05,<NA>,<NA>,<NA>,<NA>
2015-01-06,<NA>,<NA>,<NA>,<NA>
2015-01-07,<NA>,<NA>,<NA>,<NA>


In [7]:
#Train the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

df = pd.read_csv("weather.csv")

# Feature engineering
df["temp_lag1"] = df["tavg"].shift(1)
df["temp_lag7"] = df["tavg"].shift(7)
df = df.dropna()

X = df[["temp_lag1", "temp_lag7"]]
y = df["tavg"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "weather_model.pkl")
print("✅ Model trained and saved as weather_model.pkl")

✅ Model trained and saved as weather_model.pkl


In [11]:
# FastAPI with ngrok
from fastapi import FastAPI
import joblib, pandas as pd
from pyngrok import ngrok
import uvicorn
import threading
from google.colab import userdata
import os

# Load model
model = joblib.load("weather_model.pkl")

# Create FastAPI app
app = FastAPI(title="Weather Prediction API")

@app.get("/")
def home():
    return {"message": "Welcome to Weather Prediction API"}

@app.get("/predict/")
def predict(temp_lag1: float, temp_lag7: float):
    X = pd.DataFrame([[temp_lag1, temp_lag7]], columns=["temp_lag1", "temp_lag7"])
    prediction = model.predict(X)[0]
    return {"predicted_temp": round(prediction, 2)}

# Run API in background
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run_api).start()

# Get ngrok authtoken from Colab secrets
ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
if ngrok_auth_token:
    ngrok.set_auth_token(ngrok_auth_token)
else:
    print("❌ NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")

# Expose via ngrok
public_url = ngrok.connect(8000)
print("🌍 Public API URL:", public_url)

INFO:     Started server process [397]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🌍 Public API URL: NgrokTunnel: "https://a09065d759a0.ngrok-free.app" -> "http://localhost:8000"
